In [1]:
#we are taking few topics from wikipedia which will help us to make rich vocabulary in particular language with web scrapping
max_letters = 12
language_tags = {

                'en':['actor', 'cheque', 'cancer', 'chocolate', 'debate', 'hobby', 'melon', 'propaganda',
                      'religion', 'violin', 'england', 'MediaWiki'],
                'es':['Licencia','Derecho','Sociología','Interpretación','Mente','Psicología','Deseo','Placer'],

                'fr': ['Français', 'Langues_romanes', 'Charlemagne', 'Traité_de_Verdun', 'Louis_le_Pieux',
                    'Son_(physique)', 'Zoologie', 'Intelligence_animale', 'Intelligence', 'Tautologie',
                    'Pléonasme', 'Figure_de_style']

                 }

In [2]:
import re
import wikipedia as wiki
from unidecode import unidecode

#this model will perform web-scrapping and return list of words from different language
def GENERATION(tag, max_word_length):
    wiki.set_lang(tag)
    for topic in language_tags[tag]:
        page = wiki.WikipediaPage(topic)
        content = page.content
        content = unidecode(content)
        lst = CLEANING(content, max_word_length)
    return lst

In [3]:
#this model will clean all text
def CLEANING(page_content, max_word_length):
    words = re.sub(r'[^a-zA-Z ]', '', page_content)
    lower = words.lower()
    word_list = lower.split()
    short_words = []
    for word in word_list:
        if len(word) <= max_word_length:
            short_words.append(word)
    return short_words

In [4]:
#this model will return vector format of the every single words in the form of binary(0&1)
def VECTORIZATION(dic, max_word_length):
    new_list = []
    for word in dic:
        vec = ''
        n = len(word)
        for i in range(n):
            current_letter = word[i]
            ind = ord(current_letter)-97
            placeholder = (str(0)*ind) + str(1) + str(0)*(25-ind)
            vec = vec + placeholder
        if n < max_word_length:
            excess = max_word_length-n
            vec = vec +str(0)*26*excess
        new_list.append(vec)
    print(len(new_list))
    return new_list

In [5]:
def VECTOR_LABELS(tag_index, number_of_languages):
    out = str(0)*tag_index + str(1) + str(0)*(number_of_languages-1-tag_index)
    return out

In [6]:
import numpy as np
import pandas as pd

In [7]:
#upcoming code will help us to generate vectors of array that we will feed inside neural network model
word_data = []
language_data = []
master_dic = []
count = 0
for tag in language_tags.keys():
    print('generating dictionary for ' + tag)
    dic = GENERATION(tag, max_letters)
    for word in dic:
        master_dic.append(word)
    vct = VECTORIZATION(dic, max_letters)
    for vector in vct:
        word_data.append(vector)
    output_vct =VECTOR_LABELS(count, len(language_tags))
    for i in range(len(vct)):
        language_data.append(output_vct)
    count += 1

generating dictionary for en
6427
generating dictionary for es
4221
generating dictionary for fr
12514


In [8]:
arr = []
for i in range(len(word_data)):
    entry = []
    entry.append(master_dic[i])
    for digit in language_data[i]:
        entry.append(float(digit))
    for digit in word_data[i]:
        entry.append(float(digit))
    arr.append(entry)
arr = np.array(arr)
#np.save('arr.npy', arr)
df=pd.DataFrame(arr)
#df.to_csv('data.csv')

In [9]:
arr

array([['mediawiki', '1.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ['is', '1.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ['a', '1.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ...,
       ['de', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ['la', '0.0', '0.0', ..., '0.0', '0.0', '0.0'],
       ['poesie', '0.0', '0.0', ..., '0.0', '0.0', '0.0']], dtype='<U32')

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split

2022-10-29 20:14:35.328488: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
inputs = arr[:, 1+len(language_tags):]
labels = arr[:, 1:1+len(language_tags)]
x_train, x_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.15)

print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)


(3475, 312)
(3475, 3)
(19687, 312)
(19687, 3)


In [12]:
y_test=y_test.astype(np.float32)
y_train=y_train.astype(np.float32)
x_test=x_test.astype(np.float32)
x_train=x_train.astype(np.float32)

In [13]:
network = Sequential()
network.add(Dense(200, input_dim=26*max_letters, activation='sigmoid'))
network.add(Dense(150, activation='sigmoid'))
network.add(Dense(100, activation='sigmoid'))
network.add(Dense(100, activation='sigmoid'))
network.add(Dense(len(language_tags), activation='softmax'))

network.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-10-29 20:15:02.834925: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
x_test[89]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [15]:
network.fit(x_train, y_train, epochs=50,verbose=1, batch_size=1000, validation_data=(x_test, y_test))

Epoch 1/50
20/20 [==============================] - 1s 26ms/step - loss: 0.6035 - accuracy: 0.4994 - val_loss: 0.5852 - val_accuracy: 0.5396
Epoch 2/50
20/20 [==============================] - 0s 15ms/step - loss: 0.5858 - accuracy: 0.5404 - val_loss: 0.5841 - val_accuracy: 0.5396
Epoch 3/50
20/20 [==============================] - 0s 15ms/step - loss: 0.5850 - accuracy: 0.5404 - val_loss: 0.5840 - val_accuracy: 0.5396
Epoch 4/50
20/20 [==============================] - 0s 14ms/step - loss: 0.5851 - accuracy: 0.5404 - val_loss: 0.5837 - val_accuracy: 0.5396
Epoch 5/50
20/20 [==============================] - 0s 18ms/step - loss: 0.5845 - accuracy: 0.5404 - val_loss: 0.5822 - val_accuracy: 0.5396
Epoch 6/50
20/20 [==============================] - 0s 16ms/step - loss: 0.5823 - accuracy: 0.5404 - val_loss: 0.5790 - val_accuracy: 0.5396
Epoch 7/50
20/20 [==============================] - 0s 19ms/step - loss: 0.5748 - accuracy: 0.5404 - val_loss: 0.5642 - val_accuracy: 0.5396
Epoch 8/50
20

In [18]:
words=['hello','café','bonheur','Hola','Gracias']

In [28]:
def RESULT(word):
    a=0
    b=1
    while True:
        a+=1
        dic = []
        valid = False
        while not valid:
            if len(word) <= max_letters:
                word = word.lower()
                valid = True
            else:
                print('Word must be less than ' + str(max_letters + 1) + ' letters long')
        dic.append(word)
        vct_str = VECTORIZATION(dic, max_letters)
        vct = np.zeros((1, 26 * max_letters))
        count = 0
        for digit in vct_str[0]:
            vct[0,count] = int(digit)
            count += 1
        prediction_vct = network.predict(vct)

        langs = list(language_tags.keys())
        for i in range(len(language_tags)):
            lang = langs[i]
            score = prediction_vct[0][i]
            print(lang + ': ' + str(round(100*score, 2)) + '%')
        if a==b:
            break

In [30]:
RESULT('hello')

1
1/1 [==============================] - 0s 19ms/step
en: 2.16%
es: 95.28%
fr: 2.56%


In [31]:
RESULT('Gracias')

1
1/1 [==============================] - 0s 20ms/step
en: 2.65%
es: 88.99%
fr: 8.36%


In [32]:
RESULT('bonheur')

1
1/1 [==============================] - 0s 22ms/step
en: 1.65%
es: 0.97%
fr: 97.38%
